In [ ]:
!pip install pyspellchecker
!pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.8 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import ast
from spellchecker import SpellChecker
import language_tool_python

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# 加载数据
file_path = '/content/final_combined_data (1）.csv'
data = pd.read_csv(file_path)

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,article,ai_percentage
0,Imagine turning on your computer and reading s...,0
1,"Walking into the voting booth ""...every four y...",0
2,"""THE IDEA"" Venus\n\nStudying Venus is a worthy...",0
3,Body actions say more than words could ever sa...,0
4,Im going to tell about how the author of the a...,0


In [19]:
# IVAN
spell = SpellChecker()
tool = language_tool_python.LanguageTool('en-US')

# IVAN 检查语法错误数量
def check_grammar(text):
    matches = tool.check(text)
    return len(matches)

# Xiaoqian 计算特定词汇在文本中的总频率
nlp = spacy.load('en_core_web_sm')
def combined_top_word_freq(text, top_words):
    tokens = [token.text.lower() for token in nlp(text) if token.is_alpha and not token.is_stop]
    token_freq = Counter(tokens)
    return sum([token_freq.get(word, 0) for word in top_words])

# 找出AI文章中最常见的30个单词
ai_articles = data[data['ai_percentage'] == 1]
ai_text = ' '.join(ai_articles['article'])
tokens = nlp(ai_text)
filtered_tokens = [token.text.lower() for token in tokens if token.is_alpha and not token.is_stop]
word_freq = Counter(filtered_tokens)
top_30_words = [word for word, freq in word_freq.most_common(30)]
print(top_30_words)

['data', 'like', 'software', 'computer', 'system', 'systems', 'web', 'digital', 'applications', 'use', 'access', 'users', 'information', 'network', 'internet', 'online', 'including', 'user', 'security', 'language', 'devices', 'programming', 'services', 'computing', 'file', 'technology', 'content', 'include', 'different', 'making']


In [15]:
# 定义一个计算词汇多样性的函数，词汇多样性是指文本中不同词汇的比例
def lexical_diversity(text):
    tokens = word_tokenize(text) # 使用NLTK的word_tokenize函数对文本进行分词
    return len(set(tokens)) / len(tokens) if tokens else 0

# 定义一个计算平均句子长度的函数，即文本中每句话的平均词数
def avg_sentence_length(text):
    sentences = sent_tokenize(text) # 使用NLTK的sent_tokenize函数将文本分割成句子
    return sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences) if sentences else 0

# 定义一个函数来计算文本中词性标签的频率
def pos_tag_freq(text):
    tokens = word_tokenize(text) # 分词
    tags = nltk.pos_tag(tokens)  # 使用NLTK的pos_tag函数为每个词汇打上词性标签
    frequencies = FreqDist(tag for (word, tag) in tags)  # 计算不同词性标签的频率
    return dict(frequencies)

# 将上面定义的函数应用到数据集中的每篇文章上
data['lexical_diversity'] = data['article'].apply(lexical_diversity)
data['avg_sentence_length'] = data['article'].apply(avg_sentence_length)
data['pos_tag_freq'] = data['article'].apply(pos_tag_freq)

# IVAN
data['grammar_errors'] = data['article'].apply(check_grammar)

# Xiaoqian
data['combined_top_word_freq'] = data['article'].apply(lambda x: combined_top_word_freq(x, top_30_words))

# 显示数据的前几行
print(data.head())

                                             article  ai_percentage  \
0  Imagine turning on your computer and reading s...              0   
1  Walking into the voting booth "...every four y...              0   
2  "THE IDEA" Venus\n\nStudying Venus is a worthy...              0   
3  Body actions say more than words could ever sa...              0   
4  Im going to tell about how the author of the a...              0   

   lexical_diversity  avg_sentence_length  \
0           0.594340            19.272727   
1           0.414911            26.826087   
2           0.562500            24.000000   
3           0.350523            26.760000   
4           0.560847            27.000000   

                                        pos_tag_freq  grammar_errors  \
0  {'NNP': 4, 'VBG': 9, 'IN': 20, 'PRP$': 7, 'NN'...               8   
1  {'VBG': 10, 'IN': 73, 'DT': 92, 'NN': 82, '``'...              18   
2  {'``': 1, 'DT': 25, 'NNP': 14, '''': 1, 'VBZ':...              11   
3  {'NN': 187,

In [16]:
from sklearn.feature_extraction import DictVectorizer
import ast

# 检查'data' DataFrame中'pos_tag_freq'列的每个元素，如果是字符串，则将其转换为字典
data['pos_tag_freq'] = data['pos_tag_freq'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 初始化DictVectorizer，DictVectorizer用于将字典类型的数据转换为矩阵形式
vec = DictVectorizer(sparse=False)

# 使用DictVectorizer转换'data'中的'pos_tag_freq'字典为特征矩阵
pos_features = vec.fit_transform(data['pos_tag_freq'])

# 将转换后的特征矩阵创建为一个新的DataFrame
pos_feature_df = pd.DataFrame(pos_features, columns=vec.get_feature_names_out())

# 将新创建的特征DataFrame与原始的'data' DataFrame合并
data_with_pos = pd.concat([data, pos_feature_df], axis=1)

# 从合并后的DataFrame中删除原始的'pos_tag_freq'列
data_with_pos = data_with_pos.drop('pos_tag_freq', axis=1)

# 现在'data_with_pos'包含了所有原始特征和转换后的词性标签频率特征
data_with_pos.head()

,article,ai_percentage,lexical_diversity,avg_sentence_length,grammar_errors,combined_top_word_freq,#,$,'',(,...,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,``
0,Imagine turning on your computer and reading s...,0,0.594340,19.272727,8,7,0.0,0.0,2.0,0.0,...,0.0,9.0,5.0,4.0,17.0,1.0,1.0,0.0,4.0,2.0
1,"Walking into the voting booth ""...every four y...",0,0.414911,26.826087,18,2,0.0,0.0,10.0,0.0,...,3.0,10.0,11.0,11.0,32.0,3.0,3.0,0.0,5.0,8.0
2,"""THE IDEA"" Venus\n\nStudying Venus is a worthy...",0,0.562500,24.000000,11,2,0.0,0.0,1.0,0.0,...,2.0,5.0,1.0,6.0,13.0,0.0,0.0,0.0,1.0,1.0
3,Body actions say more than words could ever sa...,0,0.350523,26.760000,27,35,0.0,0.0,6.0,1.0,...,14.0,25.0,27.0,16.0,35.0,4.0,1.0,0.0,6.0,5.0
4,Im going to tell about how the author of the a...,0,0.560847,27.000000,41,4,0.0,0.0,2.0,0.0,...,4.0,16.0,7.0,6.0,20.0,0.0,1.0,0.0,5.0,5.0


In [17]:
# 选择数值型特征列
X = data_with_pos.select_dtypes(include=[np.number]).drop('ai_percentage', axis=1)
y = data_with_pos['ai_percentage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.92      0.94       209
           1       0.92      0.97      0.94       198

    accuracy                           0.94       407
   macro avg       0.94      0.94      0.94       407
weighted avg       0.94      0.94      0.94       407



In [10]:
from sklearn.model_selection import cross_val_score

# 使用交叉验证评估模型的得分
scores = cross_val_score(rf, X, y, cv=5)
print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())

Cross-validation scores: [0.95823096 0.96068796 0.93611794 0.84029484 0.8226601 ]
Mean score: 0.9035983587707724


In [11]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# 打印特征重要性
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 2 (0.266653)
2. feature 29 (0.077355)
3. feature 16 (0.047892)
4. feature 20 (0.040056)
5. feature 27 (0.038892)
6. feature 11 (0.036644)
7. feature 13 (0.034531)
8. feature 12 (0.029918)
9. feature 0 (0.027861)
10. feature 8 (0.027256)
11. feature 37 (0.025964)
12. feature 36 (0.024655)
13. feature 1 (0.024374)
14. feature 6 (0.022572)
15. feature 43 (0.022156)
16. feature 9 (0.022140)
17. feature 7 (0.021764)
18. feature 24 (0.016577)
19. feature 10 (0.016303)
20. feature 21 (0.015950)
21. feature 40 (0.015049)
22. feature 22 (0.014055)
23. feature 34 (0.013947)
24. feature 26 (0.013285)
25. feature 17 (0.012954)
26. feature 39 (0.012213)
27. feature 38 (0.011444)
28. feature 41 (0.009799)
29. feature 19 (0.009048)
30. feature 14 (0.006406)
31. feature 28 (0.006271)
32. feature 42 (0.006058)
33. feature 45 (0.005831)
34. feature 18 (0.003850)
35. feature 32 (0.003547)
36. feature 30 (0.003426)
37. feature 5 (0.002974)
38. feature 46 (0.002577)
39. feature 

In [21]:
# Load the original data set for prediction
original_data = pd.read_csv('/content/article_level_data.csv')
original_data = original_data.rename(columns = {'class' : 'ai_percentage'})
original_data = original_data.drop(columns = 'Unnamed: 0')
original_data.head()

,article,ai_percentage
0,NLP is a multidisciplinary field that draws fr...,0
1,There are a variety of emerging applications f...,0
2,As each new means of communication and social ...,0
3,"These suggestions include:, Learn about the pu...",0
4,In recent years there has been growing concern...,0


In [23]:
# 重复处理数据过程
original_data['lexical_diversity'] = original_data['article'].apply(lexical_diversity)
original_data['avg_sentence_length'] = original_data['article'].apply(avg_sentence_length)
original_data['pos_tag_freq'] = original_data['article'].apply(pos_tag_freq)

# IVAN
original_data['grammar_errors'] = original_data['article'].apply(check_grammar)

# Xiaoqian
original_data['combined_top_word_freq'] = original_data['article'].apply(lambda x: combined_top_word_freq(x, top_30_words))

original_data['pos_tag_freq'] = original_data['pos_tag_freq'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
original_pos_features = vec.transform(original_data['pos_tag_freq'])


original_pos_feature_df = pd.DataFrame(original_pos_features, columns=vec.get_feature_names_out())

# 合并
original_data_with_pos = pd.concat([original_data, original_pos_feature_df], axis=1)

original_data_with_pos = original_data_with_pos.select_dtypes(include=[np.number]).drop('ai_percentage', axis=1)

original_data_predictions = rf.predict(original_data_with_pos)

# 打印预测结果
print(original_data_predictions)

[0 0 0 ... 1 1 1]


In [24]:
true_labels = original_data['ai_percentage'].values

# 计算分类报告
report = classification_report(true_labels, original_data_predictions, target_names=['Human', 'AI'])
print(report)

              precision    recall  f1-score   support

       Human       1.00      0.97      0.98       509
          AI       0.97      1.00      0.98       509

    accuracy                           0.98      1018
   macro avg       0.98      0.98      0.98      1018
weighted avg       0.98      0.98      0.98      1018



In [25]:
sentence_data = pd.read_csv('/content/sentence_level_data.csv')
sentence_data = sentence_data.rename(columns = {'sentence': 'article', 'class' : 'ai_percentage'})
sentence_data = sentence_data.drop(columns = 'Unnamed: 0')
sentence_data.head()

,article,ai_percentage
0,NLP is a multidisciplinary field that draws fr...,0
1,"In terms of linguistics, a program must be abl...",0
2,Of course each language has its own forms of a...,0
3,Programs can use several strategies for dealin...,0
4,As formidable as the task of extracting the co...,0


In [26]:
sentence_data['lexical_diversity'] = sentence_data['article'].apply(lexical_diversity)
sentence_data['avg_sentence_length'] = sentence_data['article'].apply(avg_sentence_length)
sentence_data['pos_tag_freq'] = sentence_data['article'].apply(pos_tag_freq)

# IVAN
sentence_data['grammar_errors'] = sentence_data['article'].apply(check_grammar)

# Xiaoqian
sentence_data['combined_top_word_freq'] = sentence_data['article'].apply(lambda x: combined_top_word_freq(x, top_30_words))

sentence_data['pos_tag_freq'] = sentence_data['pos_tag_freq'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sentence_pos_features = vec.transform(sentence_data['pos_tag_freq'])

sentence_pos_feature_df = pd.DataFrame(sentence_pos_features, columns=vec.get_feature_names_out())

sentence_data_with_pos = pd.concat([sentence_data, sentence_pos_feature_df], axis=1)

sentence_data_with_pos = sentence_data_with_pos.select_dtypes(include=[np.number]).drop('ai_percentage', axis=1)

sentence_data_predictions = rf.predict(sentence_data_with_pos)

print(sentence_data_predictions)

[1 1 1 ... 1 1 1]


In [27]:
sentence_true_labels = sentence_data['ai_percentage'].values
sentence_report = classification_report(sentence_true_labels, sentence_data_predictions, target_names=['Human', 'AI'])
print(sentence_report)

              precision    recall  f1-score   support

       Human       0.95      0.26      0.41      3336
          AI       0.62      0.99      0.76      4008

    accuracy                           0.66      7344
   macro avg       0.79      0.62      0.58      7344
weighted avg       0.77      0.66      0.60      7344

